**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Optimal taxation with government consumption](#toc1_)    
- 2. [Problem 2: Labor adjustment costs](#toc2_)    
- 3. [Problem 3: Global optimizer with refined multi-start](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

## 1. <a id='toc1_'></a>[Problem 1: Optimal taxation with government consumption](#toc0_)


Consider a worker choosing hours of labor, $L\in[0,24]$, to maximize utility: 

$$
\begin{align*}
V(w,\tau,G)&=\max_{L\in[0,24]}\ln\left(C^{\alpha}G^{1-\alpha}\right)-\nu\frac{L^{2}}{2}\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}
$$

where 

* $C$ is *private* consumption with weight $\alpha\in(0,1)$.
* $\kappa > 0$ is the *free private* consumption component.
* $(1-\tau)wL$ is the *costly private* consumption component.
* $w > 0 $ is the real wage.
* $\tau \in (0,1)$ is the labor-income tax rate.
* $G > 0 $ is *government* consumption with weight $1-\alpha$.
* $\nu > 0$ is the disutility of labor scaling factor


The baseline parameters are:

$$
\begin{align*}
\alpha &= 0.5\\
\kappa &= 1.0\\
\nu &= \frac{1}{2\cdot16^2} \\
w &= 1.0 \\ 
\tau &= 0.30 \\
\end{align*}
$$

**Question 1:** Verify that the optimal labor supply choice is $L^{\star}(\tilde{w}) =\frac{-\kappa+\sqrt{\kappa^{2}+4\frac{\alpha}{\nu}\tilde{w}^2}}{2\tilde{w}}$, where $\tilde{w} = (1-\tau)w$, for $G\in\left\{1.0 , 2.0\right\}$.

In [ ]:
# a. define  as sympy symbols
alpha = sm.symbols('alpha')
kappa = sm.symbols('kappa')
nu = sm.symbols('nu')
w = sm.symbols('w')
tau = sm.symbols('tau')
C = sm.symbols('C')
G = sm.symbols('G')
L = sm.symbols('L')
wt = sm.symbols('wtilde')

# b. define utility and budget constraint
utility = sm.log(C**alpha*G**(1-alpha))-nu*(L**2/2)
budget_constraint = sm.Eq(kappa+wt*L,C)
budget_constraint_C = sm.solve(budget_constraint, C)

# c. substitute into utility and take derivative
utility_subs = utility.subs(C, budget_constraint_C[0])
foc = sm.diff(utility_subs, L)

# d. solve the FOC
sol = sm.solve(sm.Eq(foc,0), L)
sol_L = sol[1]

# e. difference to given solution
L_star = (-kappa+sm.sqrt(kappa**2+4*(alpha/nu)*wt**2))/(2*wt)
sol_func = sm.lambdify(args=(kappa,alpha,nu,wt),expr=L_star)
diff_L = sol_L-L_star

# f. compute numeric difference
diff_func = sm.lambdify(args=(kappa,alpha,nu,wt),expr=diff_L)
num_diff = diff_func(1.0,0.5,1/(2*16**2),(1-0.3)*1.0)
print(f'The difference between the given solution and the optimal solution we have found is: {num_diff:1.3f}')
print('Thus the given solution is the optimal choice')


**Question 2:** Illustrate how $L^{\star}(\tilde{w})$ depends on $w$.

In [ ]:
# a. create vectors for plot values
N = 100 # number of points to calculate
lower = 1e-8 # lower bound of w
upper = 2 # upper bound of w
w_vec = np.linspace(lower, upper, N)
L_vec = np.zeros(N)

# b. create function and set parameters
kappa = 1.0
alpha = 0.5
nu = 1/(2*16**2)
tau = 0.3

# c. calculate values
for i, w in enumerate(w_vec):
    wt = (1-tau)*w
    L_vec[i] = sol_func(kappa,alpha,nu,wt)

# d. plot figure
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(w_vec,L_vec)

# e. labels and format
ax.set_xlabel('$w$') 
ax.set_ylabel(r'$L^{*}(\tilde{w})$')
plt.title('Optimal hours to work for given wage') 
fig.tight_layout()



We now consider a government, who chooses $\tau$ and spend all of the taxes on government consumption so:

$$
G = \tau w L^{\star}((1-\tau)w)
$$

**Question 3:** Plot the implied $L$, $G$ and worker utility for a grid of $\tau$-values.


In [ ]:
# a. create vectors for plot values
N = 100 # number of points to calculate
lower = 1e-2 # lower bound of tau
upper = 1.0-lower # upper bound of tau
tau_vec = np.linspace(lower, upper, N)
L_tau_vec = np.zeros(N)
G_tau_vec = np.zeros(N)
util_tau_vec = np.zeros(N)

# b. set parameters
kappa = 1.0
alpha = 0.5
nu = 1/(2*16**2)
w = 1.0

# c. calculate values
for i, tau in enumerate(tau_vec):
    tau = tau
    wt = (1-tau)*w
    L_tau_vec[i] = sol_func(kappa,alpha,nu,wt)
    G_tau_vec[i] = tau*w*L_tau_vec[i]
    util_tau_vec[i] = np.log((kappa+(1-tau)*w*L_tau_vec[i])**alpha*G_tau_vec[i]**(1-alpha))-nu*(L_tau_vec[i]**2/2)

# d. plot figures
fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(1,3,1)
ax.plot(tau_vec,L_tau_vec)
ax.set_xlabel(r'$\tau$') 
ax.set_ylabel(r'$L^{*}(\tilde{w})$')
plt.title('Optimal hours to work for given taxrate') 
ax = fig.add_subplot(1,3,2)
ax.plot(tau_vec,G_tau_vec)
ax.set_xlabel(r'$\tau$') 
ax.set_ylabel('$G$')
plt.title('Government consumption at given taxrate')
ax = fig.add_subplot(1,3,3)
ax.plot(tau_vec,util_tau_vec)
ax.set_xlabel(r'$\tau$') 
ax.set_ylabel(r'$V(w,\tau,G)$')
plt.title('Worker utility at given taxrate')
fig.tight_layout() 

**Question 4:** Find the socially optimal tax rate $\tau^{\star}\in(0,1)$ maximizing worker utility. Illustrate your result.

In [ ]:
# a. set parameters
kappa = 1.0
alpha = 0.5
nu = 1/(2*16**2)
w = 1.0
lower = 1e-2 # lower bound of tau
upper = 1.0-lower # upper bound of tau
tau_0 = 0.5 # initial guess

# b. define negative utility dependant on tau
def negative_utility(tau):
    if tau>upper:
        neg_util=1
    elif tau<lower:
        neg_util=1
    else:
        wt = (1-tau)*w
        L = (-kappa+np.sqrt(kappa**2+4*(alpha/nu)*wt**2))/(2*wt)
        C = kappa + (1-tau)*w*L
        G = tau*w*L
        neg_util = -(np.log(C**alpha*G**(1-alpha))-nu*(L**2/2))
    return neg_util

# c. minimize negative utility
solution = optimize.minimize(negative_utility, tau_0, method='Nelder-Mead')
print(f'The socially optimal taxrate maximizing worker utility is: {solution.x[0]:1.3f}')
print(f'At this taxrate the worker utility is then: {-solution.fun:1.3f}')

# d. plot of worker utility focused around optimal taxrate
N = 100 # number of points to calculate
lower = 0.50 # lower bound of tau
upper = 0.53 # upper bound of tau
tau_opt_vec = np.linspace(lower, upper, N)
L_opt_vec = np.zeros(N)
G_opt_vec = np.zeros(N)
util_opt_vec = np.zeros(N)
for i, tau in enumerate(tau_opt_vec):
    tau = tau
    wt = (1-tau)*w
    L_opt_vec[i] = sol_func(kappa,alpha,nu,wt)
    G_opt_vec[i] = tau*w*L_opt_vec[i]
    util_opt_vec[i] = np.log((kappa+(1-tau)*w*L_opt_vec[i])**alpha*G_opt_vec[i]**(1-alpha))-nu*(L_opt_vec[i]**2/2)
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(tau_opt_vec,util_opt_vec)
ax.scatter(solution.x[0],-solution.fun, c='red', s=30)
ax.set_xlabel(r'$\tau$') 
ax.set_ylabel(r'$V(w,\tau,G)$')
plt.title('Worker utility around optimal taxrate')
fig.tight_layout() 

A more general preference formulation for the worker is:

$$
\begin{align*}
\mathcal{V}(w,\tau,G)&=\max_{L\in[0,24]}\frac{\left[ \left( \alpha C^{\frac{\sigma-1}{\sigma}}+(1-\alpha) G^{\frac{\sigma-1}{\sigma}} \right)^{\frac{\sigma}{\sigma-1} }\right]^{1-\rho}-1}{1-\rho}- \nu\frac{L^{1+\varepsilon}}{1+\varepsilon},\,\,\,\varepsilon,\rho,\sigma>0,\,\,\,\rho,\sigma\neq1\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}    
$$

Optimal labor supply is now $L^{\star}(\tilde{w},G)$.

Questions 5 and 6 must be answered with the general formulation, and for 2 different set of parameters:

- Set 1:  $\sigma = 1.001$, $\rho = 1.001$ and $\varepsilon = 1.0$.
- Set 2:  $\sigma = 1.5$, $\rho = 1.5$ and $\varepsilon = 1.0 $.

**Question 5:** Find the $G$ that solves $G = \tau w L^{\star}((1-\tau)w,G)$ using the $\tau$ found in question 4.

*Hint: First write code that solves the worker problem for given values of $G$ and $\tau$. Then find the correct G based on this.*

In [ ]:
# a. set parameters
sigma = 1.001  # set 1 value
rho = 1.01  # set 1 value
epsilon = 1.0
L_0 = 15  # initial guess

# b. define utility for given values of G and tau
def general_util_neg(L):
    C = kappa+(1-tau)*w*L
    G = tau*w*L
    util = -((((alpha*C**((sigma-1)/sigma)+(1-alpha)*G**((sigma-1)/sigma)) **
            (sigma/(sigma-1)))**(1-rho)-1)/(1-rho)-nu*(L**(1+epsilon)/(1+epsilon)))
    return util

# c. optimize given values of G and tau
tau = 0.5
sol_gvalues = optimize.minimize(general_util_neg, L_0, method='Nelder-Mead')

# d. optimize given tau from question 4 and set 1 parameters
tau = solution.x[0]
sol_gtau_set1 = optimize.minimize(general_util_neg, L_0, method='Nelder-Mead')
G_set1 = tau*w*sol_gtau_set1.x[0]

# e. optimize given tau from question 4 and set 2 parameters
sigma = 1.5  # set 2 value
rho = 1.5  # set 2 value
tau = solution.x[0]
sol_gtau_set2 = optimize.minimize(general_util_neg, L_0, method='Nelder-Mead')
G_set2 = tau*w*sol_gtau_set2.x[0]

# f. print results
print(f'Given set 1 parameters and tau from question 4 we have the optimal L: {sol_gtau_set1.x[0]:1.3f} and thus G is then: {G_set1:1.3f}')
print(f'Then when parameters are equal to set 2 values we have optimal L: {sol_gtau_set2.x[0]:1.3f} with G equal to: {G_set2:1.3f}')


**Question 6:** Find the socially optimal tax rate, $\tau^{\star}$, maximizing worker utility, while keeping $G = \tau w L^{\star}((1-\tau)w,G)$.

In [ ]:
# a. define negative utility where tau and L can be adjusted
def neg_gen_ut(x):
    L, tau = x.ravel()
    C = kappa+(1-tau)*w*L
    G = tau*w*L
    util = -((((alpha*C**((sigma-1)/sigma)+(1-alpha)*G**((sigma-1)/sigma)) **
            (sigma/(sigma-1)))**(1-rho)-1)/(1-rho)-nu*(L**(1+epsilon)/(1+epsilon)))
    return util

# b. find optimal taxrate for set 1 parameters
sigma = 1.001  # set 1 value
rho = 1.01  # set 1 value
epsilon = 1.0
t_0 = 0.5  # initial guess
L_0 = 13 # initial guess
sol_set1 = optimize.minimize(neg_gen_ut, [L_0, tau_0], method='Nelder-Mead')

# c. find optimal taxrate for set 2 parameters
sigma = 1.5  # set 1 value
rho = 1.5  # set 1 value
t_0 = 0.5  # initial guess
L_0 = 13 # initial guess
sol_set2 = optimize.minimize(neg_gen_ut, [L_0, tau_0], method='Nelder-Mead')

# d. print solutions
print(f'Given set 1 parameter values the socially optimal taxrate is: {sol_set1.x[1]:1.3f} where the workers utility is {-sol_set1.fun:1.3f} and hours worked is {sol_set1.x[0]:1.3f}')
print(f'Then when given set 2 parameter values the socially optimal taxrate is: {sol_set2.x[1]:1.3f} where the workers utility is {-sol_set2.fun:1.3f} and hours worked is {sol_set2.x[0]:1.3f}')



## 2. <a id='toc2_'></a>[Problem 2: Labor adjustment costs](#toc0_)

You own a hair salon. You employ hairdressers, $\ell_t$, to produce haircuts, $y_t = \ell_t$.

The wage for each haridresser is $w$.

The demand for haircuts implies that the price of haircuts you can charge is $p_t = \kappa_t y_t^{-\eta}$, where $\kappa_t$ is a demand-shock and $\eta \in (0,1)$ measures the elasticity of demand.

Profits are:

$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$

Baseline parameters are:
- $\eta = 0.5$
- $w = 1.0$

**Question 1:** Verify numerically that $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ maximises profits, for $\kappa\in\left\{1.0 , 2.0\right\}$.

In [ ]:
# write your code here

We now consider a *dynamic* version of the model.

* The demand-shock is a so-called AR(1) in logs, 

$$
\log \kappa_{t} = \rho \log \kappa_{t-1} + \epsilon_{t},\,\,\, \epsilon_{t+1} \sim \mathcal{N}(-0.5\sigma_{\epsilon}^2,\sigma_{\epsilon})
$$

* Any hiring or firing implies a fixed adjustment cost, $\iota > 0 $.
* Future profits are discounted with a monthly factor of $R \in (0,1)$.

The initial demand shock is $\kappa_{-1} = 1$ and the planning horizon is 10 years, i.e. 120 months so $t \in \{0,1,2,\dots,119\}$. Initially you don't have any employees, $\ell_{-1}=0$


The *ex post* value of the salon is *conditional* on the shock series is:

$$
h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119}) = \left[\sum_{t=0}^{119}R^{-t}\left[\kappa_{t}\ell_{t}^{1-\eta}-w\ell_{t}-\boldsymbol{1}_{\ell_{t}\neq\ell_{t-1}}\iota\right]\right]
$$

The *ex ante* expected value of the salon can be approximated by

$$
H = \mathbb{E}[h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119})] \approx \frac{1}{K}\sum_{k=0}^{K} h(\epsilon_0^k,\epsilon_1^k,\dots,\epsilon_{119}^k)
$$

where each $k\in\{0,1,\dots,K-1\}$ is a random shock series. Maximizing profitability means maximizing $H$.


Baseline parameters are: 

- $\rho = 0.90$
- $\iota = 0.01$
- $\sigma_{\epsilon} = 0.10$
- $R = \left(1+0.01\right)^{1/12}$

**Question 2:** Calculate $H$ if the policy  $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ from question 1 is followed. Choose $K$ so the approximation is good enough to not affect your results substantially.

In [ ]:
# write your code here 

Next, we consider policies on the form:

$$

\ell_{t}=\begin{cases}
\ell_t^{\ast}  & \text{if }\left|\ell_{t-1}-\ell_t^{\ast} \right|>\Delta\\
\ell_{t-1} & \text{else }
\end{cases}
\\
\text{where}\,\,\ell_t^{\ast} = \left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}} \\

$$
With $\Delta \geq 0$ and $\Delta = 0$ being the previous policy.



**Question 3:** Calculate $H$ if the policy above was followed with $\Delta = 0.05$. Does it improve profitability?

In [ ]:
# write your code here

**Question 4:** Find the optimal $\Delta$ maximizing $H$. Illustrate your result.

In [ ]:
# write your code here


**Question 5:** Suggest an alternative policy you believe might improve profitability. Implement and test your policy.



In [ ]:
# write your code here

## 3. <a id='toc3_'></a>[Problem 3: Global optimizer with refined multi-start](#toc0_)

We consider the Griewank function:

$$ f(\boldsymbol{x}) = \sum^n_{i=1} \frac{x^2_i}{4000}-\prod^n_{i=1}\cos\left(\frac{x_i}{\sqrt{i}}\right)+1$$

The **global minimum** of this function is $f(0,0) = 0$ (remember: $\cos(0)=1$).<br>
But the function also have a lot of **local minima**.

In [ ]:
def griewank(x):
    return griewank_(x[0],x[1])
    
def griewank_(x1,x2):
    A = x1**2/4000 + x2**2/4000
    B = np.cos(x1/np.sqrt(1))*np.cos(x2/np.sqrt(2))
    return A-B+1

A **refined global optimizer with multi-start** is:

1. Choose *bounds* for $\mathbf{x}$ and *tolerance* $\tau > 0$.
2. Choose number of *warm-up iterations*, $\underline{K} > 0$ and *maximum number of iterations*, $K > \underline{K}$.
3. In each iteration for $k \in \{0,1,\dots,K-1\}$:

    A. Draw random $\mathbf{x}^k$ uniformly within chosen bounds.

    B. If $k < \underline{K}$ go to step E.

    C. Calculate $\chi^k = 0.50\cdot\frac{2}{1+\exp((k-\underline{K})/100)}$  

    D. Set $\mathbf{x}^{k0} = \chi^k \mathbf{x}^k + (1-\chi^k)\mathbf{x}^{\ast} $

    E. Run optimizer with $\mathbf{x}^{k0}$ as initial guess and $\mathbf{x}^{k\ast}$ as result.

    F. Set $\mathbf{x}^{\ast} = \mathbf{x}^{k\ast}$ if $k = 0$ or $f(\mathbf{x}^{k\ast}) < f(\mathbf{x}^{\ast})$

    G. If $f(\mathbf{x}^{\ast}) < \tau$ go to step 4.

4. Return the result $\mathbf{x}^{\ast}$.

As settings we choose:

* $x_1,x_2 \in  [-600,600]$
* $\tau = 10^{-8}$
* $\underline{K}=10$
* $K=1000$

The optimizer in Step 3.E is `BFGS` with a tolerance of $\tau$.

**Question 1:** Implement the refined global optimizer with multi-start. Illustrate how the effective initial guesses $\mathbf{x}^{k0}$ vary with the iteration counter $k$.

In [ ]:
#chatgbt

import numpy as np

def griewank(x):
    return griewank_(x[0], x[1])

def griewank_(x1, x2):
    A = x1**2 / 4000 + x2**2 / 4000
    B = np.cos(x1 / np.sqrt(1)) * np.cos(x2 / np.sqrt(2))
    return A - B + 1

def optimize_griewank(bounds, tolerance, K_1, K):
    x_star = None

    for k in range(K):
        # Step 3A: Generate random x^k uniformly within the chosen bounds
        x_k = np.random.uniform(bounds[0], bounds[1], size=2)

        if k < K_1:
            # Step 3B: Skip to step E if k < K_1
            x_k0 = x_k
        else:
            # Step 3C: Calculate chi^k
            chi_k = 0.50 * (2 / (1 + np.exp((k - K_1) / 100)))

            # Step 3D: Compute x^(k0)
            x_k0 = chi_k * x_k + (1 - chi_k) * x_star

        # Step 3E: Run optimizer with x^(k0) as initial guess and store the result in x^(k*)
        x_k_star = optimizer(x_k0)  # Replace 'optimizer' with the appropriate optimization method

        # Step 3F: Update the best solution x^*
        if k == 0 or griewank(x_k_star) < griewank(x_star):
            x_star = x_k_star

        # Step 3G: Check the termination condition
        if griewank(x_star) < tolerance:
            break

    # Step 4: Return the best solution x^*
    return x_star


In [427]:
# 1. choose bounds x and tolerance tau > 0
x1 = -600
x2 = 600
tau = 10**(-8)

# 2. choose number of warm-up iterations K_ > 0 and maximum number of iterations K > K_
K_ = 10
K = 1000

# 3. 
# A. draw random x^k uniformly within bounds
x=[]
k=0
while k < K:
   x.append(np.random.uniform(x1,x2))
   k += 1
# B. if k < K_ go to E
xk_star=[]
x0=np.ones(K)*x[0]

for k,x in enumerate(x):
   if k < K_:
      res=optimize.minimize(griewank,x0,method='BFGS',tol=tau)
      xk_star.append(res.x[0])
      if k==0:
         x_star=xk_star
      elif griewank(xk_star) < griewank(x_star):
         x_star=xk_star
      print(xk_star)
# C. calculate xk
   elif k >= K_:
      xk=0.5*2/(1+np.exp((k-K_)/100))
# D. set xk0
      xk0=np.ones(K)*(xk*x+(1-xk)*x_star)
# E. run optimizer
      res2=optimize.minimize(griewank,xk0,method='BFGS',tol=tau)
      xk_star[k]=res2.x[0]
# F. set x_star 
      if k==0 or griewank(xk_star) < griewank(x_star):
         x_star=xk_star

# G. go to step 4         
         if griewank(x_star) < tau:
            break   

# 4. return x_star
print(x_star)


[-562.0304048070566]
[-562.0304048070566, -562.0304048070566]
[-562.0304048070566, -562.0304048070566, -562.0304048070566]
[-562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566]
[-562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566]
[-562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566]
[-562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566]
[-562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566]
[-562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566, -562.0304048070566]


KeyboardInterrupt: 

**Question 2:** Is it a better idea to set $\underline{K} = 100$? Is the convergence faster?

In [ ]:
# write your code here